# Workshop 4 -- Transfer Learning -- Head Model

Change to a particular version of TensorFlow, which is later supported with on-device transfer learning.

In [1]:
!pip install tensorflow==2.0.0rc0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 86.3MB 71kB/s 
     |████████████████████████████████| 501kB 44.0MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 4.3MB 39.5MB/s 
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
2.0.0-rc0


Mount Google Drive to access the generic model.

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Head Model

Download the Transfer Learning converter (will be stored in the Runtime).

In [3]:
!wget https://github.com/osaukh/mobile_computing_lab/raw/master/code/OnDeviceActivityRecognition/converter.zip
!unzip converter

--2021-04-25 15:09:51--  https://github.com/osaukh/mobile_computing_lab/raw/master/code/OnDeviceActivityRecognition/converter.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/osaukh/mobile_computing_lab/master/code/OnDeviceActivityRecognition/converter.zip [following]
--2021-04-25 15:09:51--  https://raw.githubusercontent.com/osaukh/mobile_computing_lab/master/code/OnDeviceActivityRecognition/converter.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61401 (60K) [application/zip]
Saving to: ‘converter.zip’

converter.zip       100%[===================>]  59.96K  --.-KB/s    in 0.001s  

20

Import the necessary python packages and load the model. Chop off generic model's head (up to the layer names "headlayer").

In [4]:
import sys
sys.path.insert(0,'converter')

from tensorflow.keras.models import Sequential, Model, load_model, save_model
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

model_m = load_model('/content/gdrive/My Drive/MCL/WS04/saved_model.pbtxt')
# chop off model's head
model = Model(model_m.input, model_m.get_layer('headlayer').output)
print(model.summary())
save_model(model, '/content/gdrive/My Drive/MCL/WS04/chopped_model.pbtxt', 
           include_optimizer = False,save_format="tf")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_input (InputLayer)   [(None, 600)]             0         
_________________________________________________________________
reshape (Reshape)            (None, 200, 3)            0         
_________________________________________________________________
dense (Dense)                (None, 200, 32)           128       
_________________________________________________________________
dropout (Dropout)            (None, 200, 32)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 200, 16)           528       
_________________________________________________________________
headlayer (Dense)            (None, 200, 8)            136       
Total params: 792
Trainable params: 792
Non-trainable params: 0
_______________________________________________________________

Prepare a new head.

In [5]:
# New head configuration
num_classes = 2
learning_rate = 0.001
batch_size = 5
l2_rate = 0.0001
hidden_units = 6
input_shape = model.get_layer('headlayer').output.shape

head = tf.keras.Sequential([
    layers.Flatten(input_shape=(1, 200, 8)),
    layers.Dense(
        units=hidden_units,
        activation="relu",
        kernel_regularizer=l2(l2_rate)),
    layers.Dense(
        units=num_classes,
        activation="softmax",
        kernel_regularizer=l2(l2_rate)),
])
print(head.summary())

# Optimizer is ignored by the converter.
head.compile(loss="categorical_crossentropy", optimizer="adam")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 9606      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 14        
Total params: 9,620
Trainable params: 9,620
Non-trainable params: 0
_________________________________________________________________
None


Call TF converter to prepare the on-device transfer leaning models. This should be executed once. If you have to re-run this, reset your runtime.

In [6]:
base = bases.SavedModelBase('/content/gdrive/My Drive/MCL/WS04/chopped_model.pbtxt')

converter = TFLiteTransferConverter(num_classes,
                base,
                heads.KerasModelHead(head),
                optimizers.SGD(learning_rate),
                train_batch_size=batch_size)

converter.convert_and_save('/content/gdrive/My Drive/MCL/WS04/custom_keras_model')

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatu

/usr/local/lib/python3.7/dist-packages/tensorflow_core/lite/python/lite.py:854: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)
